In [12]:
import tensorflow as tf
print(tf.version)
print(tf.config.list_physical_devices('GPU'))

<module 'tensorflow._api.v2.version' from 'c:\\Users\\roopa\\miniconda3\\envs\\tensorflow\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [13]:
train_dataset=tf.keras.utils.image_dataset_from_directory("archive/train",image_size=(48,48),color_mode="grayscale",batch_size=32)
val_dataset=tf.keras.utils.image_dataset_from_directory("archive//test",image_size=(48,48),color_mode="grayscale",batch_size=32)
print(train_dataset.class_names)


Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [14]:
normalization_layer=tf.keras.layers.Rescaling(1./255)
#normalizes pixel values to 0-1
train_dataset=train_dataset.map(lambda x,y:(normalization_layer(x),y))
val_dataset=val_dataset.map(lambda x,y:(normalization_layer(x),y))
#applies normalization to batches of image aka x and y(unchanged i.e labels of class)

In [15]:
#add agumentation
data_agumentation=tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])
train_dataset=train_dataset.map(lambda x,y:(data_agumentation(x),y))

In [16]:
#adding prefetch to data so gpu doesnt remain idle-prepares a batch of data in ram
AUTOTUNE=tf.data.AUTOTUNE
train_dataset=train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset=val_dataset.prefetch(buffer_size=AUTOTUNE)

In [17]:
#initializing model
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(48,48,1)),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256,activation="relu"),
    tf.keras.layers.Dense(units=256,activation="relu"),
    tf.keras.layers.Dense(units=7,activation="softmax"),
])

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss',   # track validation loss
    patience=10,           # stop if no improvement after 5 epochs
    restore_best_weights=True  # rollback to best model
)


In [19]:
model.compile(optimizer='adam',loss="sparse_categorical_crossentropy",metrics=['accuracy'])

In [ ]:
print(tf.config.list_physical_devices('GPU'))
model.fit(x=train_dataset,validation_data=val_dataset,epochs=50,callbacks=[early_stop])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Epoch 1/50
898/898 [==============================] - 69s 75ms/step - loss: 1.7808 - accuracy: 0.2659 - val_loss: 1.7108 - val_accuracy: 0.3036
Epoch 2/50
898/898 [==============================] - 33s 37ms/step - loss: 1.6405 - accuracy: 0.3425 - val_loss: 1.5282 - val_accuracy: 0.4089
Epoch 3/50
898/898 [==============================] - 36s 40ms/step - loss: 1.5488 - accuracy: 0.3909 - val_loss: 1.4400 - val_accuracy: 0.4452
Epoch 4/50
898/898 [==============================] - 35s 39ms/step - loss: 1.4738 - accuracy: 0.4279 - val_loss: 1.4078 - val_accuracy: 0.4579
Epoch 5/50
898/898 [==============================] - 40s 44ms/step - loss: 1.4177 - accuracy: 0.4524 - val_loss: 1.3455 - val_accuracy: 0.4720
Epoch 6/50
898/898 [==============================] - 38s 43ms/step - loss: 1.3772 - accuracy: 0.4687 - val_loss: 1.3431 - val_accuracy: 0.4896
Epoch 7/50
898/898 [==============================] - 33s 36ms/step -

In [21]:
model.save('emotion.h5')